In [15]:
import pandas as pd
import glob
import os


ML = True #False : DL 
SUB_AGG = False #FALSE : sub-wise 

def get_path_to_results(ML, SUB_AGG):
    path_to_results = './results/'
    path_to_results += 'ML/' if ML else 'DL/'
    path_to_results += 'sub_agg/' if SUB_AGG else 'sub_wise/'
    return path_to_results

#separate into aggregate and take_stat
def aggregate_and_take_stat(result_csv_root_path,
                            result_file_name_prefix="results_seed-",
                            seed_ranges=(0,49),
                            metrics=['accuracy', 'roc_auc', 'f1_score', 'balanced_accuracy'],
                            sub_agg = False):
    if sub_agg : 
        #! TODO ;IMPLMENET THE SUBWISE AGGREGATION (FOR LOOP OVER SUBS OR STH AND COMBINE THE COMBINED_DF OR STH!)
        #use aggregate_results and take_stats functions here
        combined_df = aggregate_seed_results(result_csv_root_path,
                            result_file_name_prefix=result_file_name_prefix,
                            seed_ranges=seed_ranges)
        print(combined_df.shape)
    else:
        combined_df_across_subs = []
        sub_dirs = glob.glob(result_csv_root_path + '/subject-*/')
        for sub_dir in sub_dirs:
            sub_path = sub_dir
            combined_df_sub = aggregate_seed_results(sub_path,
                            result_file_name_prefix=result_file_name_prefix,
                            seed_ranges=seed_ranges)
            combined_df_across_subs.append(combined_df_sub)
        combined_df = pd.concat(combined_df_across_subs, ignore_index=True)
        print(combined_df.shape)
        
            
    assert combined_df is not None, "No results found"
    result_df = take_stats(combined_df, metrics=metrics)
    return result_df


def take_stats(df, group_by_cols = ['model_name', 'data_name'],
               metrics=['accuracy', 'roc_auc', 'f1_score', 'balanced_accuracy']):
    """
    Calculate mean, std, min, max for each metric, after grouping by group_by_cols
    """
    # Group and calculate statistics
    grouped = df.groupby(group_by_cols)
    # Calculate means and stds
    stats_dict = {}
    for metric in metrics:
        stats_dict[f'{metric}_mean'] = grouped[metric].mean()
        stats_dict[f'{metric}_std'] = grouped[metric].std()
        # stats_dict[f'{metric}_min'] = grouped[metric].min()
        # stats_dict[f'{metric}_max'] = grouped[metric].max()
    
    # Create final DataFrame
    result_df = pd.DataFrame(stats_dict).reset_index()
    
    return result_df.sort_values(group_by_cols)
    
    
def aggregate_seed_results(result_csv_root_path,
                        result_file_name_prefix="results_seed-",
                        seed_ranges=(0,49)):    
    # Read and combine all files
    df_list = []
    for seed in range(seed_ranges[0], seed_ranges[1]+1):
        file_path = os.path.join(result_csv_root_path,
                                f"{result_file_name_prefix}{seed}.csv")
        if os.path.exists(file_path):
            df_list.append(pd.read_csv(file_path))
    
    assert len(df_list) == seed_ranges[1] - seed_ranges[0] + 1, f"Some files are missing, expected {seed_ranges[1] - seed_ranges[0] + 1} csv files, got {len(df_list)}"
        
    combined_df = pd.concat(df_list, ignore_index=True)
    return combined_df      
    

#test the function
path_to_results = get_path_to_results(ML, SUB_AGG)
print(path_to_results)
aa = aggregate_and_take_stat(path_to_results, seed_ranges=(0, 49), sub_agg = SUB_AGG)


./results/ML/sub_wise/
(120000, 6)


In [16]:
#df[(df['model_name'] == 'SVC') & (df['data_name'] == 'base')]
#! TODO : add step to remove NuSVC. as it didn't work for sub3

In [17]:
aa #'./results/ML/sub_agg/'


,model_name,data_name,accuracy_mean,accuracy_std,roc_auc_mean,roc_auc_std,f1_score_mean,f1_score_std,balanced_accuracy_mean,balanced_accuracy_std
0,DecisionTreeClassifier,PCA_10-base,0.512407,0.080233,0.480844,0.083423,0.506377,0.164482,0.480844,0.083423
1,DecisionTreeClassifier,PCA_10-fixation_denormed,0.507963,0.087625,0.479065,0.086779,0.505649,0.156668,0.479065,0.086779
2,DecisionTreeClassifier,PCA_10-fixation_normalized,0.523333,0.073604,0.492421,0.074409,0.525614,0.143591,0.492421,0.074409
3,DecisionTreeClassifier,PCA_10-halfnfft,0.507778,0.080206,0.477044,0.084004,0.502973,0.171674,0.477044,0.084004
4,DecisionTreeClassifier,PCA_10-halfoverlap,0.500370,0.090626,0.465013,0.089040,0.502915,0.165415,0.465013,0.089040
...,...,...,...,...,...,...,...,...,...,...
795,SVC,halfoverlap_halfnfft,0.540185,0.087103,0.471447,0.063190,0.563789,0.203863,0.471447,0.063190
796,SVC,nooverlap,0.527222,0.090310,0.457210,0.063446,0.550395,0.215286,0.457210,0.063446
797,SVC,nooverlap_halfnfft,0.527222,0.090310,0.457210,0.063446,0.550395,0.215286,0.457210,0.063446
798,SVC,whole_trial_denormed,0.555370,0.070838,0.477886,0.059332,0.514266,0.259379,0.477886,0.059332


In [18]:
aa[-80:-50]

,model_name,data_name,accuracy_mean,accuracy_std,roc_auc_mean,roc_auc_std,f1_score_mean,f1_score_std,balanced_accuracy_mean,balanced_accuracy_std
720,Ridge,PCA_5-base,0.610000,0.080834,0.498078,0.040505,0.537922,0.301858,0.498078,0.040505
721,Ridge,PCA_5-fixation_denormed,0.621111,0.078256,0.508595,0.042085,0.552538,0.287862,0.508595,0.042085
722,Ridge,PCA_5-fixation_normalized,0.611667,0.075301,0.499013,0.045115,0.538356,0.311398,0.499013,0.045115
723,Ridge,PCA_5-halfnfft,0.610926,0.080359,0.498944,0.040199,0.538204,0.301676,0.498944,0.040199
724,Ridge,PCA_5-halfoverlap,0.609444,0.081558,0.497880,0.041853,0.537527,0.302244,0.497880,0.041853
725,Ridge,PCA_5-halfoverlap_halfnfft,0.609815,0.081055,0.498281,0.041057,0.538162,0.301408,0.498281,0.041057
726,Ridge,PCA_5-nooverlap,0.612593,0.077558,0.501560,0.042419,0.540386,0.301704,0.501560,0.042419
727,Ridge,PCA_5-nooverlap_halfnfft,0.612593,0.077558,0.501560,0.042419,0.540386,0.301704,0.501560,0.042419
728,Ridge,PCA_5-whole_trial_denormed,0.621667,0.076947,0.508637,0.040569,0.555248,0.286217,0.508637,0.040569
729,Ridge,PCA_5-whole_trial_normalized,0.615556,0.073955,0.510507,0.048611,0.534644,0.315194,0.510507,0.048611


In [19]:
aa = aa.sort_values(['roc_auc_mean','accuracy_mean'], ascending=False)

In [20]:
aa[:50]



,model_name,data_name,accuracy_mean,accuracy_std,roc_auc_mean,roc_auc_std,f1_score_mean,f1_score_std,balanced_accuracy_mean,balanced_accuracy_std
140,GradientBoostingClassifier,base,0.553148,0.080065,0.520750,0.081979,0.542755,0.174000,0.520750,0.081979
43,DecisionTreeClassifier,halfnfft,0.550370,0.077130,0.518611,0.080724,0.544990,0.166389,0.518611,0.080724
281,LinearSVC,PCA_50-fixation_denormed,0.522778,0.079359,0.518412,0.079401,0.530996,0.108318,0.518412,0.079401
288,LinearSVC,PCA_50-whole_trial_denormed,0.519630,0.085544,0.517643,0.086574,0.526052,0.110472,0.517643,0.086574
144,GradientBoostingClassifier,halfoverlap,0.561296,0.084677,0.516425,0.082305,0.552013,0.192170,0.516425,0.082305
40,DecisionTreeClassifier,base,0.543333,0.079581,0.516334,0.080973,0.532462,0.177321,0.516334,0.080973
44,DecisionTreeClassifier,halfoverlap,0.540370,0.089684,0.514184,0.089548,0.535269,0.169564,0.514184,0.089548
143,GradientBoostingClassifier,halfnfft,0.548333,0.082705,0.513315,0.085182,0.537753,0.183237,0.513315,0.085182
289,LinearSVC,PCA_50-whole_trial_normalized,0.518704,0.082333,0.513275,0.083383,0.531870,0.106776,0.513275,0.083383
558,MLP64,PCA_10-whole_trial_denormed,0.550741,0.080569,0.513166,0.078012,0.612861,0.128455,0.513166,0.078012


In [23]:
best_rows = aa.loc[aa.groupby('model_name')['roc_auc_mean'].idxmax()]
best_rows.sort_values('roc_auc_mean', ascending=False)


,model_name,data_name,accuracy_mean,accuracy_std,roc_auc_mean,roc_auc_std,f1_score_mean,f1_score_std,balanced_accuracy_mean,balanced_accuracy_std
140,GradientBoostingClassifier,base,0.553148,0.080065,0.520750,0.081979,0.542755,0.174000,0.520750,0.081979
43,DecisionTreeClassifier,halfnfft,0.550370,0.077130,0.518611,0.080724,0.544990,0.166389,0.518611,0.080724
281,LinearSVC,PCA_50-fixation_denormed,0.522778,0.079359,0.518412,0.079401,0.530996,0.108318,0.518412,0.079401
558,MLP64,PCA_10-whole_trial_denormed,0.550741,0.080569,0.513166,0.078012,0.612861,0.128455,0.513166,0.078012
329,MLP128,PCA_5-whole_trial_normalized,0.542593,0.092025,0.511809,0.084468,0.525085,0.158621,0.511809,0.084468
229,LinearRegression,PCA_5-whole_trial_normalized,0.615741,0.074049,0.510785,0.048909,0.534741,0.315264,0.510785,0.048909
729,Ridge,PCA_5-whole_trial_normalized,0.615556,0.073955,0.510507,0.048611,0.534644,0.315194,0.510507,0.048611
535,MLP512,PCA_50-halfoverlap_halfnfft,0.546481,0.092184,0.510480,0.085213,0.558198,0.166975,0.510480,0.085213
468,MLP32,PCA_20-whole_trial_denormed,0.500926,0.104254,0.509792,0.080246,0.395947,0.139994,0.509792,0.080246
416,MLP256,PCA_20-nooverlap,0.539630,0.085046,0.509626,0.074305,0.552887,0.164175,0.509626,0.074305


In [9]:
aa[aa['model_name']=="MLP128"]

,model_name,data_name,accuracy_mean,accuracy_std,roc_auc_mean,roc_auc_std,f1_score_mean,f1_score_std,balanced_accuracy_mean,balanced_accuracy_std
417,MLP128,PCA_20-nooverlap_halfnfft,0.529259,0.043845,0.542763,0.042286,0.495072,0.078737,0.542763,0.042286
415,MLP128,PCA_20-halfoverlap_halfnfft,0.525556,0.044052,0.540318,0.042632,0.499681,0.085037,0.540318,0.042632
416,MLP128,PCA_20-nooverlap,0.523333,0.040909,0.537924,0.040974,0.502442,0.084203,0.537924,0.040974
414,MLP128,PCA_20-halfoverlap,0.523704,0.043677,0.537471,0.043026,0.499668,0.091327,0.537471,0.043026
410,MLP128,PCA_20-base,0.520370,0.042739,0.534947,0.043396,0.500319,0.080565,0.534947,0.043396
424,MLP128,PCA_5-halfoverlap,0.530556,0.047481,0.533813,0.041922,0.527516,0.126778,0.533813,0.041922
420,MLP128,PCA_5-base,0.528333,0.048109,0.533583,0.043911,0.527215,0.112706,0.533583,0.043911
413,MLP128,PCA_20-halfnfft,0.521296,0.041461,0.532742,0.042734,0.501841,0.085975,0.532742,0.042734
427,MLP128,PCA_5-nooverlap_halfnfft,0.529630,0.044974,0.531081,0.043281,0.536349,0.115156,0.531081,0.043281
425,MLP128,PCA_5-halfoverlap_halfnfft,0.529259,0.051105,0.527959,0.043236,0.551178,0.125058,0.527959,0.043236
